In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import copy
import os
import yaml
import pickle
from datetime import timedelta

import sys
sys.path.append('../../')

from data.processing.processing import generate_simulated_data
from data.dataloader import SimulatedDataLoader

from main.seir.fitting import single_fitting_cycle
from utils.generic.config import read_config
from utils.fitting.loss import Loss_Calculator
from utils.fitting.smooth_jump import smooth_big_jump

from viz.data import plot_data
from utils.generic.enums.columns import *
from utils.fitting.smooth_jump import perform_smoothing_experiment
from viz.utils import setup_plt, axis_formatter

# Smoothing Exp

In [ ]:
config_filename = 'simulate_data.yaml'
config = read_config(config_filename)

In [ ]:
seed = 132
num_trials = 7000

method = 'weighted-mag'

In [ ]:
np.random.seed(seed)
results = []
for i in range(num_trials):
    results.append(perform_smoothing_experiment(config, method))

In [ ]:
print(method)
print(np.array(results).mean(axis=0), 'mean')
print(np.array(results).std(axis=0), 'std')

print(np.median(results, axis=0), 'median')
print(median_abs_deviation(results, axis=0), 'mad')

In [ ]:
1.56819117/np.sqrt(5000)

# Plotting

In [ ]:
with open('../../../paper/smoothing/plotting/data_sim.pkl', 'rb+') as pickle_file:
    df_dict = pickle.load(pickle_file)

In [ ]:
df = df_dict['df']
df_spike = df_dict['df_spike']
df_spike_smooth = df_dict['df_spike_smooth']

In [ ]:
plt.rcParams.update({
    'text.usetex': True,
    'font.size': 20,
    'font.family': 'Palatino',
 })

In [ ]:
from data.processing.processing import get_data_from_tracker

In [ ]:
config_filename = 'default.yaml'
config1 = read_config(config_filename)

df1 = get_data_from_tracker(state='Maharashtra', district='Mumbai', use_dataframe='data_all')['data_frame']
df1 = df1[df1['date'] <= '2020-07-07']

# Perform smoothing
print('smoothing params', config1['fitting']['data']['smooth_jump_params'])
df1_smooth, _ = smooth_big_jump(df1, config1['fitting']['data']['smooth_jump_params'])

In [ ]:
mumbai_dict = {'df_mumbai' : df1, 'df_mumbai_smooth' : df1_smooth}

with open('../../../paper/smoothing/plotting/data_mumbai.pkl', 'wb+') as pickle_file:
    pickle.dump(mumbai_dict, pickle_file)

In [ ]:
def format_func(value, tick_number):
    if value >= 1000:
        return f"{int(value/1000)}k"
    else:
        return f"{int(value)}"

In [ ]:
fig, axs = plt.subplots(figsize=(10, 7), nrows=2, ncols=2)
for comp in ['active', 'recovered']:
    if comp == 'active':
        plot_ax = axs.flat[0]
    else:
        plot_ax = axs.flat[2]
    compartment = Columns.from_name(comp)
    plot_ax.plot(df[compartments['date'][0].name].to_numpy(), df[compartment.name].to_numpy(),
                 '--', color=compartment.color)
    plot_ax.plot(df_spike[compartments['date'][0].name].to_numpy(), df_spike[compartment.name].to_numpy(),
                 '-o', color=compartment.color, ms=3)
    plot_ax.plot(df_spike_smooth[compartments['date'][0].name].to_numpy(), df_spike_smooth[compartment.name].to_numpy(),
                 '-', color=compartment.color)
#     plot_ax.tick_params(axis='y', labelcolor=compartment.color)
    plot_ax.set_ylabel(compartment.name.title())

legend_elements = [
    Line2D([0], [0], ls='--', color='black', label='Unspiked'),
    Line2D([0], [0], ls='-', marker='o', ms=3, color='black', label='Spiked'),
    Line2D([0], [0], ls='-', color='black', label='Smoothed')
]
first_legend = axs.flat[0].legend(handles=legend_elements, loc='lower center', fontsize=18)
axs.flat[0].add_artist(first_legend)

for comp in ['active', 'recovered']:
    compartment = Columns.from_name(comp)
    if comp == 'active':
        plot_ax = axs.flat[1]
    else:
        plot_ax = axs.flat[3]
    
    plot_ax.plot(df1[compartments['date'][0].name].to_numpy(), df1[compartment.name].to_numpy(),
                 '-o', ms=3, color=compartment.color)
    plot_ax.plot(df1_smooth[compartments['date'][0].name].to_numpy(), df1_smooth[compartment.name].to_numpy(),
                 '-', color=compartment.color)
#     plot_ax.tick_params(axis='y', labelcolor=compartment.color)
    
legend_elements = [
    Line2D([0], [0], ls='-', marker='o', ms=3, color='black', label='Original'),
    Line2D([0], [0], ls='-', color='black', label='Smoothed')
]
first_legend = axs.flat[3].legend(handles=legend_elements, loc='upper left', fontsize=18)
axs.flat[3].add_artist(first_legend)

for ax in axs.flat:
    axis_formatter(ax)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
# fig.tight_layout()

for ax in axs[0, :]:
    ax.xaxis.set_ticklabels([])
    
plt.subplots_adjust(hspace=0.05)

axs.flat[2].set_xlabel('Simulated', fontsize=20)
axs.flat[3].set_xlabel('Mumbai', fontsize=20)

axs.flat[2].tick_params(axis='x', labelsize=15)
axs.flat[3].tick_params(axis='x', labelsize=15)

import matplotlib.dates as mdates
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU

axs.flat[3].xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=FR, interval=3))

fig.savefig(f'../../../paper/plots/smoothing-combined.pdf', format='pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df1_smooth

In [ ]:
from datetime import datetime

In [ ]:
datetime.strptime('2020-09-19', '%Y-%m-%d') + timedelta(days=28)

In [ ]:
smooth_big_jump(df1, config1['fitting']['data']['smooth_jump_params'])